---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

### Importing an excel file

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
filepath = '../data/Energy+Indicators.xls'
xl = pd.ExcelFile(filepath)

In [3]:
# parsing first (index: 0) sheet
total_rows = xl.book.sheet_by_index(0).nrows

In [4]:
header = [16,17]
rows = 245 
footer = total_rows - rows

In [5]:
energy = pd.read_excel('../data/Energy+Indicators.xls', header=header, skipfooter=footer)

### Cleaning columns

In [6]:
# join multiindex in tuple with '_'
energy.columns = energy.columns.map('_'.join)

In [7]:
# drop unnecessary columns 
col_drop = [energy.columns[0], energy.columns[2]]
energy.drop(columns=col_drop, inplace=True)

In [8]:
# rename columns
energy.rename(columns={energy.columns[0]:'Country'}, inplace=True)
energy = energy.rename(columns=lambda x: x.lower().replace(' ','_'))

In [9]:
# confirm changes
energy.tail()

,country,energy_supply_petajoules,energy_supply_per_capita_gigajoules,renewable_electricity_production_%
222,Viet Nam,2554,28,45.32152
223,Wallis and Futuna Islands,0,26,0.00000
224,Yemen,344,13,0.00000
225,Zambia,400,26,99.71467
226,Zimbabwe,480,32,52.53612


Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

### Cleaning dataset as per the instruction

In [10]:
# convert petajoules to gigajoules
petajoule_to_gigajoules = 1000000
energy['energy_supply_petajoules'] = energy['energy_supply_petajoules'] * petajoule_to_gigajoules

In [11]:
energy.rename(columns={energy.columns[1]: 'energy_supply',
                       energy.columns[2]: 'energy_supply_per_capita'}
             ,inplace=True)

In [12]:
# extract indices for incorrect entries (i.e. ...)
incorrect_entry = energy[(energy['energy_supply'].str.isdigit() == False) | (energy['energy_supply_per_capita'].str.isdigit() == False)].index

In [82]:
incorrect_entry

Int64Index([3, 86, 150, 210, 217], dtype='int64')

In [13]:
# change incorrect data to np.nan
energy.loc[incorrect_entry, 'energy_supply'] = energy.loc[incorrect_entry, 'energy_supply'].apply(lambda x: np.nan)
energy.loc[incorrect_entry, 'energy_supply_per_capita'] = energy.loc[incorrect_entry, 'energy_supply_per_capita'].apply(lambda x:np.nan)

In [14]:
# change to numeric datatypes
energy['energy_supply'] = energy['energy_supply'].astype('float')
energy['energy_supply_per_capita'] = energy['energy_supply_per_capita'].astype('float')
energy['renewable_electricity_production_%'] = energy['renewable_electricity_production_%'].astype('float')

In [15]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 4 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   country                             227 non-null    object 
 1   energy_supply                       222 non-null    float64
 2   energy_supply_per_capita            222 non-null    float64
 3   renewable_electricity_production_%  227 non-null    float64
dtypes: float64(3), object(1)
memory usage: 7.2+ KB


#### Replace values

In [18]:
# testing regex for unicode 
test= ["Curaçao", "Côte d'Ivoire", "Bolivia (Plurinational State of)", "Switzerland1234"]
pattern = re.compile(r"([^\(\d]+)", re.U)
[re.match(pattern, w).group().strip() for w in test]

['Curaçao', "Côte d'Ivoire", 'Bolivia', 'Switzerland']

In [19]:
# use regex to keep only alphabet + whitespace + ' + - in 'country'
pattern = re.compile(r"([^\(\d]+)", re.U)
energy['country'] = energy['country'].apply(lambda x: re.match(pattern, x).group(1)).unique()
energy['country'] = energy['country'].str.strip()

In [ ]:
# replace method to change values in a dataframe
country_name_to_change = {"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"}

energy['country'].replace(country_name_to_change, inplace=True)

In [ ]:
# confirm changes
energy.query('country in ["South Korea", "United States", "United Kingdom", "Hong Kong"]')

In [21]:
energy['country']

0                    Afghanistan
1                        Albania
2                        Algeria
3                 American Samoa
4                        Andorra
                 ...            
222                     Viet Nam
223    Wallis and Futuna Islands
224                        Yemen
225                       Zambia
226                     Zimbabwe
Name: country, Length: 227, dtype: object

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

In [22]:
GDP = pd.read_csv('../data/world_bank.csv', skiprows=4)
GDP = GDP.drop(columns=['Indicator Name', 'Indicator Code'])

In [23]:
# change column names lowercase, without whiteplace
GDP.rename(columns=lambda x: x.lower().replace(' ','_'), inplace=True)

In [24]:
GDP.rename(columns={'country_name':'country'}, inplace=True)

In [25]:
country_to_replace={"Korea, Rep.": "South Korea", 
                    "Iran, Islamic Rep.": "Iran",
                    "Hong Kong SAR, China": "Hong Kong"}

GDP['country'].replace(country_to_replace, inplace=True)

In [26]:
# confirm changes
GDP.query('country in ["South Korea","Iran","Hong Kong"]')

,country,country_code,1960,1961,1962,1963,1964,1965,1966,1967,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
93,Hong Kong,HKG,NaN,NaN,NaN,NaN,NaN,1.764691e+10,1.796359e+10,1.825160e+10,...,2.019164e+11,2.149698e+11,2.195440e+11,2.141452e+11,2.286377e+11,2.396459e+11,2.437206e+11,2.512081e+11,2.577757e+11,2.638606e+11
109,Iran,IRN,5.500820e+10,6.072406e+10,6.552689e+10,7.014134e+10,7.608598e+10,8.904776e+10,9.929040e+10,1.104634e+11,...,3.895523e+11,4.250646e+11,4.289909e+11,4.389208e+11,4.677902e+11,4.853309e+11,4.532569e+11,4.445926e+11,4.639027e+11,NaN
123,South Korea,KOR,2.757696e+10,2.893825e+10,2.964966e+10,3.247578e+10,3.493136e+10,3.674434e+10,4.141087e+10,4.393695e+10,...,9.410199e+11,9.924316e+11,1.020510e+12,1.027730e+12,1.094499e+12,1.134796e+12,1.160809e+12,1.194429e+12,1.234340e+12,1.266580e+12


Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [27]:
ScimEn = pd.read_excel('../data/scimagojr-3.xlsx')
ScimEn.rename(columns=lambda x: x.lower().replace('-','_').replace(' ','_'), inplace=True)

In [28]:
# slicing ScimEn dataframe
top15_ScimEn = ScimEn[ScimEn['rank'] <= 15]

In [30]:
# slicing GDP dataframe
GDP10 = GDP.iloc[:, np.r_[0:2, len(GDP.columns)-10:len(GDP.columns)]]

In [51]:
df = pd.merge(left=top15_ScimEn, right=energy, how='inner', left_on='country', right_on='country')

In [54]:
df = pd.merge(left=df, right=GDP10, how='inner', left_on='country', right_on='country')

In [78]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, China to Brazil
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Rank                      15 non-null     int64  
 1   Documents                 15 non-null     int64  
 2   Citable documents         15 non-null     int64  
 3   Citations                 15 non-null     int64  
 4   Self-citations            15 non-null     int64  
 5   Citations per document    15 non-null     float64
 6   H index                   15 non-null     int64  
 7   Energy Supply             15 non-null     float64
 8   Energy Supply per Capita  15 non-null     float64
 9   % Renewable               15 non-null     float64
 10  2006                      15 non-null     float64
 11  2007                      15 non-null     float64
 12  2008                      15 non-null     float64
 13  2009                      15 non-null     float64
 14  2010     

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.75491,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.57098,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.23282,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.60047,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.28868,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12


### Change the column names again as per the instruction

In [72]:
df_columns = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']

In [74]:
print(len(df_columns) == len(df.columns))
df.columns

True


Index(['rank', 'documents', 'citable_documents', 'citations', 'self_citations',
       'citations_per_document', 'h_index', 'energy_supply',
       'energy_supply_per_capita', 'renewable_electricity_production_%',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015'],
      dtype='object')

In [59]:
df.set_index('country', inplace = True)

In [63]:
df.drop(columns='country_code', inplace = True)

In [75]:
df.columns = df_columns

In [79]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, China to Brazil
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Rank                      15 non-null     int64  
 1   Documents                 15 non-null     int64  
 2   Citable documents         15 non-null     int64  
 3   Citations                 15 non-null     int64  
 4   Self-citations            15 non-null     int64  
 5   Citations per document    15 non-null     float64
 6   H index                   15 non-null     int64  
 7   Energy Supply             15 non-null     float64
 8   Energy Supply per Capita  15 non-null     float64
 9   % Renewable               15 non-null     float64
 10  2006                      15 non-null     float64
 11  2007                      15 non-null     float64
 12  2008                      15 non-null     float64
 13  2009                      15 non-null     float64
 14  2010     

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.75491,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.57098,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.23282,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.60047,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.28868,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12


---

*This function should return a DataFrame with 20 columns and 15 entries.*

In [80]:
def answer_one():
    return df.shape

In [81]:
answer_one()

(15, 20)

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [34]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [35]:
def answer_two():
    return "ANSWER"

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [36]:
def answer_three():
    Top15 = answer_one()
    return "ANSWER"

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [37]:
def answer_four():
    Top15 = answer_one()
    return "ANSWER"

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [38]:
def answer_five():
    Top15 = answer_one()
    return "ANSWER"

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [39]:
def answer_six():
    Top15 = answer_one()
    return "ANSWER"

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [40]:
def answer_seven():
    Top15 = answer_one()
    return "ANSWER"

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [41]:
def answer_eight():
    Top15 = answer_one()
    return "ANSWER"

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [42]:
def answer_nine():
    Top15 = answer_one()
    return "ANSWER"

In [43]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [44]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [45]:
def answer_ten():
    Top15 = answer_one()
    return "ANSWER"

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [46]:
def answer_eleven():
    Top15 = answer_one()
    return "ANSWER"

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [47]:
def answer_twelve():
    Top15 = answer_one()
    return "ANSWER"

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [48]:
def answer_thirteen():
    Top15 = answer_one()
    return "ANSWER"

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [49]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [50]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!